In [ ]:
import numpy as np
import json
import requests
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, WMTSTileSource, LogColorMapper

In [ ]:
output_notebook()

In [ ]:
data = requests.get("https://data.lacity.org/resource/v87k-wgde.json").json()

In [ ]:
values = [float(item["fy_12_13"]) for item in data]
locations = np.array([item["location_1"]["coordinates"] for item in data])
zipcodes = [item["location_1_zip"] for item in data]

In [ ]:
from pyproj import Proj, transform

from_proj = Proj(init="epsg:4326")
to_proj = Proj(init="epsg:3857")

def convert(longitudes, latitudes):
    x_vals = []
    y_vals = []
    for lon, lat in zip(longitudes, latitudes):
        x, y = transform(from_proj, to_proj, lon, lat)
        x_vals.append(x)
        y_vals.append(y)
    return x_vals, y_vals

In [ ]:
x,y = convert(locations[:,0], locations[:,1])

In [ ]:
fig = figure(title="Los Angeles Water Usage 2012-2013", plot_width=600, plot_height=600, tools=["wheel_zoom", "pan", "hover"],
            x_range=(-13209490, -13155375), y_range=(3992960, 4069860), 
             webgl=True, active_scroll="wheel_zoom")
fig.axis.visible = False

STAMEN_TONER_BACKGROUND = WMTSTileSource(
    url='http://tile.stamen.com/toner-background/{Z}/{X}/{Y}.png',
    attribution=(
        'Map tiles by <a href="http://stamen.com">Stamen Design</a>, '
        'under <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a>.'
        'Data by <a href="http://openstreetmap.org">OpenStreetMap</a>, '
        'under <a href="http://www.openstreetmap.org/copyright">ODbL</a>'
    )
)

background = fig.add_tile(STAMEN_TONER_BACKGROUND)

In [ ]:
color_mapper = LogColorMapper(palette="RdYlGn10", low=4, high=75)

source = ColumnDataSource(data=dict(x=x, y=y, size=6*np.log(values), values=values, zipcodes=zipcodes))

fig.circle('x', 'y', size='size', source=source, line_alpha=0.0, alpha=.7, fill_color={'field':'values', 'transform': color_mapper})
fig.select_one(HoverTool).tooltips = [
    ('Usage', '@values'),
    ('Zip Code', '@zipcodes')
]

In [ ]:
show(fig)